In [1]:
def show_usage():
    print("usage: python nfs-mount.py -f <vip file> -v <view> -m <mount>")
    print(" -f : file that has vip list")
    print(" -v : Cohesity view")
    print(" -m : mount-prefix (like /coh/ora)")

In [6]:
import re
import os
import sys
import argparse
import subprocess
from subprocess import call

In [10]:
argv = sys.argv[1:]
argv = [i.lower() for i in argv]
if '--help' in argv or 'help' in argv or '-help' in argv:
    show_usage()
    print("exit")
    exit(0)
parser = argparse.ArgumentParser(description = "Parser for nfs-mount", conflict_handler='resolve')

parser.add_argument("-f", "--vipfile", help = "file that has vip list", 
                    required = True, default = "")
parser.add_argument("-v", "--view", help = "Cohesity view", 
                    required = True, default = "")
parser.add_argument("-m", "--mount", help = "mount-prefix (like /mnt/ora)",
                    required = True, default = "")

try:
    args = parser.parse_args(argv)
except argparse.ArgumentError:
    show_usage()
    
if (os.path.exists(args.vipfile)):
    print("file {} provided exist, script continue".format(args.vipfile))
else:
    print("file {} provided does not exist".format(args.vipfile))
    
i = 1
vipfile = open(args.vipfile, "r")
Lines = vipfile.readlines() 
for line in Lines:
    if (os.path.isdir("{0}{1}".format(mount, i)) == False):
        print("Directory {0}{1} does not exist, create it".format(mount, i))
        try:
            os.system("mkdir -p {0}{1}".format(mount, i))
        except OSError:
            print("Exception created with create new directory {0}{1}".format(mount, i))
            exit(1)
    temp = ""
    line = line.strip()
    print("Check whether IP {} can be connected".format(line))
    if (line != "" and len(line) != 0):
        temp = "/bin/ping {} -c 2".format(line)
        cmd = ["/bin/ping", "{}".format(line), "-c", "2"]
        proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        o, e = proc.communicate()
        output = o.decode('ascii')
        if(len(e.decode('ascii'))== 0):
            output = output.splitlines()
        else:
            print(e)
            print("Error when running ps -ef")
            exit(2)
        result = []
        for x in output:
            if re.search("error", x):
                result.append(x)
        if (len(result)!=0):
            print("error: IP {} can't be connected. It may not be a valid IP. Skip this IP".format(line))
        else:
            print("IP {} can be connected".format(line))
            print("check whether the mount point is in /etc/fstab")
            mpreturn = []
            preturn = []
            ## mpreturn=`grep -i $ip /etc/fstab | grep -i $view`
            file = open("/etc/fstab", "r")
            for x in file:
                if re.search(line, x):
                    preturn.append(x)
            for x in preturn:
                if re.search("{}".format(view), x):
                    mpreturn.append(x)
            if(len(mpreturn) == 0):
                print("{}{} is not in /etc/fstab, add it to /etc/fstab".format(mount, i))
                file = open("/etc/fstab", "a")
                file.write("{}:/{} {}{} nfs intr,hard,rsize=1048576,wsize=1048576,proto=tcp,vers=3 0 0".format(line, view, mount, i))
            else:
                print("{}{} is already in /etc/fstab".format(mount, i))
                
            if os.path.ismount("{0}{1}".format(mount, i)):
                print("{0}{1} is mount point".format(mount, i))
            else:
                os.mount("{0}{1}".format(mount, i))
    i=i+1
os.system("ls -ld {}1".format(mount))
print("Need to change the permission to Oracle if it is not set up yet")


NameError: name 'sys' is not defined